In [1]:
import os
os.chdir('..')

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from tqdm import tqdm




/var/folders/m5/9j_7j_8j5m154hyxw8j2346h0000gn/T/ipykernel_52548/3062625772.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
# load packages
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score, classification_report

import torch
import torch.nn.functional as F
from torch.utils import data
from torchinfo import summary
import torch.nn as nn
import torch.optim as optim

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
xtx_data = pd.read_pickle('data/data.pkl')
target_col = 'y'

In [5]:
def prepare_x(data):
    df1 = data[:40, :].T
    return np.array(df1)

def get_label(data):
    lob = data[-5:, :].T
    return lob

def prepare_x(data):
    return data.drop('y', axis=1).values

def get_label(data):
    return data.y.values

def data_classification(X, Y, T):
    [N, D] = X.shape
    df = np.array(X)

    dY = np.array(Y)

    dataY = dY[T - 1:N]

    dataX = np.zeros((N - T + 1, T, D))
    for i in trange(T, N + 1):
        dataX[i - T] = df[i - T:i, :]

    return dataX, dataY

def torch_data(x, y):
    x = torch.from_numpy(x)
    x = torch.unsqueeze(x, 1)
    y = torch.from_numpy(y)
    y = F.one_hot(y, num_classes=3)
    return x, y

In [6]:
class Dataset(data.Dataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, k, num_classes, T):
        """Initialization""" 
        self.k = k
        self.num_classes = num_classes
        self.T = T
            
        x = prepare_x(data)
        y = get_label(data)
        x, y = data_classification(x, y, self.T)
        y = y[:,self.k] - 1
        self.length = len(x)

        x = torch.from_numpy(x)
        self.x = torch.unsqueeze(x, 1)
        self.y = torch.from_numpy(y)

    def __len__(self):
        """Denotes the total number of samples"""
        return self.length

    def __getitem__(self, index):
        """Generates samples of data"""
        return self.x[index], self.y[index]

In [7]:
# dec_data = np.loadtxt('other_data/Train_Dst_NoAuction_DecPre_CF_7.txt')
# dec_train = dec_data[:, :int(np.floor(dec_data.shape[1] * 0.8))]
# dec_val = dec_data[:, int(np.floor(dec_data.shape[1] * 0.8)):]

# dec_test = np.loadtxt('other_data/Test_Dst_NoAuction_DecPre_CF_7.txt')


In [8]:
N = xtx_data.shape[0] * 3 // 4
dec_train = xtx_data[:N]
dec_test = xtx_data[N:]

In [10]:
batch_size = 64

dataset_train = Dataset(data=dec_train, k=4, num_classes=3, T=100)
dataset_val = Dataset(data=dec_val, k=4, num_classes=3, T=100)
dataset_test = Dataset(data=dec_test, k=4, num_classes=3, T=100)

train_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=dataset_val, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=False)

print(dataset_train.x.shape, dataset_train.y.shape)


 65%|██████▍   | 1693620/2623150 [00:18<00:15, 58189.96it/s] 

: 

: 

In [13]:
tmp_loader = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=1, shuffle=True)

for x, y in tmp_loader:
    print(x)
    print(y)
    print(x.shape, y.shape)
    break

tensor([[[[0.1305, 0.0007, 0.1303,  ..., 0.1008, 0.1294, 0.0150],
          [0.1305, 0.0007, 0.1303,  ..., 0.1008, 0.1294, 0.0150],
          [0.1305, 0.0007, 0.1303,  ..., 0.1008, 0.1294, 0.0150],
          ...,
          [0.1306, 0.0358, 0.1304,  ..., 0.0107, 0.1295, 0.0100],
          [0.1306, 0.0358, 0.1304,  ..., 0.0107, 0.1295, 0.0100],
          [0.1306, 0.0537, 0.1304,  ..., 0.0107, 0.1295, 0.0100]]]],
       dtype=torch.float64)
tensor([0.], dtype=torch.float64)
torch.Size([1, 1, 100, 40]) torch.Size([1])


In [16]:
x = prepare_x(dec_train)
y = get_label(dec_train)
x, y = data_classification(x, y, 10)

In [18]:
x.shape

(203791, 10, 40)